### 查看GPU

In [1]:
!nvidia-smi

Tue Apr 16 15:46:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### 安裝需要的BERT套件

In [2]:
!pip install transformers==4.29.0 -U
!pip install nlp -U
!pip install torch==2.0.0 -U
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 62.7 MB/s eta 0:00:00
 

### 載入套件

In [3]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from nlp import load_dataset, Dataset
import nlp
import torch
import random
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

### 抓取訓練資料

In [4]:
!wget https://github.com/shhuangmust/AI/raw/112-1/train.csv

--2024-04-16 15:50:04--  https://github.com/shhuangmust/AI/raw/112-1/train.csv
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shhuangmust/AI/112-1/train.csv [following]
--2024-04-16 15:50:04--  https://raw.githubusercontent.com/shhuangmust/AI/112-1/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388895 (380K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 379.78K  --.-KB/s    in 0.02s   

2024-04-16 15:50:04 (17.4 MB/s) - ‘train.csv’ saved [388895/388895]



### 讀取訓練資料集

In [5]:
df = pd.read_csv('train.csv')
dataset = Dataset.from_pandas(df)

In [6]:
df.sample(10)

,ID,Name,Description,label
125,125,《挺出事業線》紅石榴濃縮錠,「只要14天輕鬆擊退暗沉、擺脫平胸女…醫師豐胸推薦、童顏巨乳…3天有感-7天見效-14天給妳...,1
788,788,輔酵素Q10-NK,輔酵素Q10(CoQ10)分布於人體全身，但隨著年齡增加，體內合成輔酵素Q10的濃度會逐漸減...,0
769,769,靈芝菌絲體錠(含β-聚葡萄糖)(Usana),以專利技術從麵包酵母菌(Saccharomycescerevisiae)提取獨特結構β-1，...,0
698,698,活力寶纖飲品(美樂家),足量6克膳食纖維，就像每天吃到5蔬果，更添加最佳洋車前子纖維，結合水溶性及非水溶性纖維，幫助...,0
215,215,「花蓮郭火腿御采禮盒(600g二條入/盒)」、「[伴手禮專案]花蓮郭火腿_煙燻火腿2入(90...,「[伴手禮專案]花蓮郭火腿_煙燻火腿2入(900g/入)」「抗氧化劑)」,1
483,483,美麗神纖即纖草本飲,「『泡芙族』看起來不胖，皮下脂肪卻過多…特徵:肌肉量和骨質密度的嚴重不足，導致體力較差、易疲...,1
179,179,印加果油,「協助降三高，預防併發...促進腸胃蠕動...改善體脂...提高精力，修復腦細胞...過敏大...,1
605,605,【GNC健安喜】優卓美佳女用複方維他命90錠(女性綜合維他命/美肌加強版),GNC專為女性設計，添加美顏配方，加強鈣、鐵、維他命B群等女性所需的綜合營養素。添加20多種...,0
395,395,蜂王乳錠,「吃出好肌底...『癸烯酸』擁有強力的抗菌效果，亦被稱為天然抗生素...補充天然雌激素......,1
710,710,活力思衛錠(美樂家),擷取地中海健康飲食概念配方，含6大天然營養素，含維生素A/C/E及葡萄籽/橄欖果/番茄萃取物...,0


### 載入原始模型

In [7]:
model = BertForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm-ext')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

###  載入分詞模型

In [8]:
tokenizer = BertTokenizerFast.from_pretrained('hfl/chinese-bert-wwm-ext')

tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

### 設定執行參數

In [9]:
RANDOM_SEED = 5
MAX_LEN = 512
EPOCHS=5
BATCH_SIZE = 8

### 把資料集分成訓練、測試、驗證

In [10]:
shuffled_ds = dataset.shuffle(RANDOM_SEED)
split_ds = shuffled_ds.train_test_split(test_size=0.2)
train_dataset = split_ds['train']
test_val_dataset = split_ds['test']
split_tv = test_val_dataset.train_test_split(test_size=0.5)
test_dataset = split_tv['train']
val_dataset = split_tv['test']

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

### 取一筆出來瞧瞧(已經打亂)

In [11]:
train_dataset[2]

{'Description': '：「全新3倍力……它是全世界登上美國第1名抗過敏益生菌就是宏醫敏素（宏醫敏元素）……敏元素3倍力幫您同時滅火，PFAR菌，P菌幫助眼睛鼻子所有過敏，降發炎，滅喉嚨氣管發火，滅肺部發火；R菌是全身免疫性、腦部滅火，很多人在腦部莫名偏頭痛、開始憂鬱、莫名發怒……腦部在過敏發炎，這個一定要R菌才有效果；F菌滅皮膚發炎；A菌滅腸胃發炎……（輔導案例）他嚴重到耳朵斷裂，皮膚炎、氣喘……每天早上1包，晚上1包，15天後它所有紅腫癢全消（使用前後圖示）；4歲開始吃，他每天吃敏元素長大，比別人高1個頭；斯容大蕁麻疹，吃這1支很快好，她3天整個（症狀）全消；有人最快40分鐘，紅腫癢全消……你現在有任何的症狀，你有腸胃敏感、皮膚、鼻子、夏天很容易口角破的、有腸病毒的、拉肚子、腹瀉，我告訴你，它真的2包見效…（主持人）我自己蕁麻疹發作，12點45分發作，我吃2包敏元素，13點30分蕁麻疹全退……能抗過敏、抗發炎，全世界就只有這1支……裡面有4種菌種……」「敏感掰掰……抗發炎功效及改善免疫系統」「鳳梨酵素抗發炎專利……抗發炎功效及改善免疫系統」',
 'ID': 529,
 'Name': '宏醫敏元素體質平衡特效組/宏醫敏元素3倍力敏感掰掰組/超級有酵宏醫敏元素3倍力單盒',
 'label': 1}

In [12]:
tokenizer.tokenize("體重跟體脂都有在持續緩慢的下降中")

['體',
 '重',
 '跟',
 '體',
 '脂',
 '都',
 '有',
 '在',
 '持',
 '續',
 '緩',
 '慢',
 '的',
 '下',
 '降',
 '中']

### 定義分詞器

In [13]:
def tokenize(batch):
  return tokenizer(batch['Description'], max_length=MAX_LEN, padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

### 定義評測準則，訓練器

In [14]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=50,
    weight_decay=0.01,
    # evaluate_during_training=True,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

### 開始訓練

In [15]:
trainer.train()
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


{'eval_loss': 0.06991709768772125,
 'eval_accuracy': 0.9893617021276596,
 'eval_f1': 0.991869918699187,
 'eval_precision': 1.0,
 'eval_recall': 0.9838709677419355,
 'eval_runtime': 3.1756,
 'eval_samples_per_second': 29.6,
 'eval_steps_per_second': 3.779,
 'epoch': 5.0}

### 找一個範例來測試

In [16]:
print(df.iloc[220]['label'], df.iloc[220]['Description'])
print(df.iloc[578]['label'], df.iloc[578]['Description'])

1 「增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常」
0 ·???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品。


### 設定測試文字

In [17]:
#test_text = '增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常'
test_text = '???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品'

MAX_LEN = 512

encoded_review = tokenizer.encode_plus(
    test_text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    return_attention_mask=True,
    return_tensors='pt',
    truncation=True
)

In [18]:
encoded_review

{'input_ids': tensor([[  101,   136,   136,   136,   136,   136,   136,   136,  3680,   819,
          1419,  3300,  8114,  8882,  6737, 11382,   511,  6737, 11382,  1377,
          1347,  5645,  5204,  4495,  5162,   147,  1086,  4495,  8024,  1060,
          5442,  1072,  1295,  1398,   868,  4500,  8024,  1377,   914,  6868,
          3173,  7376,   807,  6342,   511,   924,  6362,  2542,  4472,  7518,
          3268,  5204,  2898,   978,  2434,   511,  3268,  7077,  3189,  3315,
          4638,  3297,   881,  5401,  2159,  5469,  1501,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

### 進行測試

In [19]:
test_model = model.to('cuda')

input_ids = encoded_review['input_ids'].to('cuda')
attention_mask = encoded_review['attention_mask'].to('cuda')
output = test_model(input_ids, attention_mask)
result = torch.argmax(output[0][0])

classnames = ['合格', '不合格']

print(f'廣告詞: {test_text}')
print(f'檢測結果: {classnames[result]}')

廣告詞: ???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品
檢測結果: 合格
